In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.pipeline import Pipeline  
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression

# Step 1: Load and preprocess data
outcomes = pd.read_csv('outcomes.csv')
projects = pd.read_csv('projects.csv')
donations = pd.read_csv('donations.csv')

# Map 't' → 1 and 'f' → 0 in 'fully_funded'
outcomes['fully_funded'] = outcomes['fully_funded'].map({'t': 1, 'f': 0})
outcomes['not_fully_funded'] = 1 - outcomes['fully_funded']  # Alternative to lambda

# Merge datasets and filter by date
projects['date_posted'] = pd.to_datetime(projects['date_posted'])

# Merge datasets
data = projects.merge(outcomes, on='projectid', how='inner').merge(
    donations[['projectid', 'donation_timestamp', 'donation_total']], on='projectid', how='left')

# Define feature types
numeric_features = ['students_reached', 'total_price_excluding_optional_support', 'donation_total']
categorical_features = ['primary_focus_subject', 'resource_type', 'primary_focus_area']

# ✅ **Define Transformations (Place Before Model Pipeline!)**
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# ✅ **Now Create Model Pipeline (After Defining Preprocessor!)**
model_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LogisticRegression(max_iter=500, solver='liblinear', random_state=42))
])

# Train model
X_train = data[numeric_features + categorical_features]
y_train = data['fully_funded']

model_pipeline.fit(X_train, y_train)

data = data[data['date_posted'] >= '2010-01-01']

# Confirm the conversion
print(outcomes['not_fully_funded'].value_counts())
print(outcomes[['projectid','is_exciting', 'not_fully_funded']].head())

# Add new time-related features 
data['year_posted'] = data['date_posted'].dt.year
data['month_posted'] = data['date_posted'].dt.month
data['days_since_posted'] = (pd.Timestamp.now() - data['date_posted']).dt.days

# Add interaction features
data['state_poverty_interaction'] = data['school_state'] + '_' + data['poverty_level']


In [ ]:
# Step 2: Minimal feature matrix
features = [
    'primary_focus_subject', 
    'school_state', 
    'resource_type', 
    'poverty_level',
    'year_posted',
    'month_posted',
    'days_since_posted',
    'state_poverty_interaction'
]
X = pd.get_dummies(data[features], drop_first=True)
y = data['not_fully_funded']

# Step 3: Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.80, random_state=42)

# Step 4: Train model and predict probabilities
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_proba = rf.predict_proba(X_test)[:, 1]


# Step 5: Rank probabilities and calculate precision
results_df = pd.DataFrame({'score': y_proba})
results_df['rank'] = results_df['score'].rank(method='first', ascending=False)

# Label top 10% as 1 and calculate precision
top_10_percent_cutoff = int(len(results_df) * 0.1)
results_df['prediction'] = (results_df['rank'] <= top_10_percent_cutoff).astype(int)
precision = precision_score(y_test, results_df['prediction'])

# Output results
print(f"Precision for top 10%: {precision}")
print(results_df.head(15))

In [ ]:
from sklearn.linear_model import LogisticRegression

# Replace RandomForest with Logistic Regression
model_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LogisticRegression(max_iter=500, solver='liblinear', random_state=42))
])

# Train model
X_train = data[numeric_features + categorical_features]
y_train = data['fully_funded']
model_pipeline.fit(X_train, y_train)

# Get probabilities for ranking
y_proba = model_pipeline.predict_proba(X_train)[:, 1]  # Extract probability of being fully funded
results_df = pd.DataFrame({'score': y_proba})
results_df['rank'] = results_df['score'].rank(method='first', ascending=False)

# Define the top 10% threshold for precision calculation
top_10_percent_cutoff = int(len(results_df) * 0.1)
results_df['prediction'] = (results_df['rank'] <= top_10_percent_cutoff).astype(int)

# Calculate precision
from sklearn.metrics import precision_score
precision = precision_score(y_train, results_df['prediction'])

# Output results
print(f"Precision for top 10%: {precision}")
print(results_df.head(15))


In [ ]:
# Step 5: Rank probabilities and calculate precision
results_df = pd.DataFrame({
    'projectid': data.loc[X_test.index, 'projectid'], # Match project ID from data_sample
    'score': y_proba # Predicted probability of not getting funded
})
# Sort by score in descending order
results_df = results_df.sort_values(by='score', ascending=False)

# Assign rank based on score order 
results_df['rank'] = results_df['score'].rank(method='first', ascending=False)

# Label top 10% as needing review 
threshold = results_df['score'].quantile(0.90)  # Top 10% based on actual probability distribution
results_df['prediction'] = (results_df['score'] >= threshold).astype(int)

# Compute precision score 
precision = precision_score(y_test, results_df['prediction'])

# Output results
print(f"Precision for top 10%: {precision}")
print(results_df.head(15))

In [ ]:
#import matplotlib.pyplot as plt

#importance = rf.feature_importances_
#plt.figure(figsize=(12, 10))  # Increase figure size for clarity
#plt.barh(X_train.columns, importance)  # Plot feature importance
#plt.xticks(rotation=45, fontsize=10)  # Rotate text to prevent overlap
#plt.ylabel("Features")  # Label for Y-axis
#plt.xlabel("Importance Score")  # Label for X-axis
#plt.title("Feature Importance")  # Add a meaningful title
#plt.show()


#top_features = pd.Series(importance, index=X_train.columns).nlargest(20)  # Show top 20 features
#top_features.plot(kind='barh', figsize=(12, 8))
#plt.title("Top 20 Most Important Features")
#plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Retrieve model coefficients
logistic_coef = model_pipeline.named_steps['model'].coef_[0]

# Retrieve feature names after transformation
feature_names = model_pipeline.named_steps['preprocessor'].get_feature_names_out()

# Create a DataFrame for better visualization
coef_df = pd.DataFrame({'Feature': feature_names, 'Importance': logistic_coef})
coef_df = coef_df.sort_values(by='Importance', ascending=False)  # Sort by importance

# Display top features
plt.figure(figsize=(12, 8))
plt.barh(coef_df['Feature'].head(20), coef_df['Importance'].head(20))  # Show top 20 features
plt.xlabel("Coefficient Value")
plt.ylabel("Features")
plt.title("Top 20 Most Important Features in Logistic Regression")
plt.show()